# EDA of Patient Data
# 02_EDA_Patient

| Date | User | Change Type | Remarks |  
| ---- | ---- | ----------- | ------- |
| 28/09/2025 | Adrienne | Created   | Moved Martin's code to this program | 
| |  |  | |

# Content

* [Introduction](#introduction)

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime
import json_lines
import seaborn as sns

In [11]:
path = "../data/clean"
#claim = pd.read_pickle(f"{path}/claim.pkl")
claim = pd.read_pickle(f"{path}/claim_mini_sample.pkl")
patient = pd.read_pickle(f"{path}/patient.pkl")
eob = pd.read_pickle(f"{path}/eob.pkl")
# coverage = pd.read_pickle(f"{path}/coverage.pkl")

In [10]:
print(patient.columns)

Index(['address_0_city', 'address_0_line', 'address_0_postalCode',
       'address_0_state', 'birthDate', 'deceasedDateTime', 'extension_0_url',
       'extension_0_valueCode', 'extension_1_url',
       'extension_1_valueCoding_code', 'extension_1_valueCoding_display',
       'extension_1_valueCoding_system', 'extension_2_extension_0_url',
       'extension_2_extension_0_valueCoding_code',
       'extension_2_extension_0_valueCoding_display',
       'extension_2_extension_0_valueCoding_system',
       'extension_2_extension_1_url', 'extension_2_extension_1_valueString',
       'extension_2_url', 'extension_3_url', 'extension_3_valueDate',
       'extension_4_url', 'extension_4_valueCoding_code',
       'extension_4_valueCoding_system', 'extension_5_url',
       'extension_5_valueCoding_code', 'extension_5_valueCoding_system',
       'extension_6_url', 'extension_6_valueCoding_code',
       'extension_6_valueCoding_system', 'extension_7_url',
       'extension_7_valueCoding_code', 'exte

In [12]:
# Select relevant columns for initial exploration
patient = patient[['address_0_state', 'birthDate', 'deceasedBoolean', 'extension_1_valueCoding_display', 'extension_3_valueDate', 'gender', 'id', 'identifier_1_value', 'deceasedBoolean', 'deceasedDateTime', 'address_0_postalCode']]
patient = patient.rename({
  'extension_1_valueCoding_display': 'race',
  'extension_3_valueDate': 'referenceYear',
  'identifier_1_value': 'patient_medicare_number',
  'address_0_postal_code': 'postalCode'
}, axis=1)

### Exploring a Patient

In [13]:
patient_id = -10000000000322
medicare_number = "1S00E00AA50"

In [16]:
c = claim[claim['patient_medicare_number'] == medicare_number]
e = eob[eob['patient_number'] == str(patient_id)]
p = patient[patient['patient_medicare_number'] == medicare_number]

In [23]:
c.unique_claim_ID.head()

32504         100005174
32434         100007096
32467         100005066
32521    22729615336f16
32473         100005075
Name: unique_claim_ID, dtype: object

In [17]:

# Checking if there are any non-na DRGs with HCPCS codes
c['hcpcs_0'] = c['hcpcs_ls'].str[0]
c[(~c['drg_code'].isna()) & (~c['hcpcs_0'].isna())]

/var/folders/h7/vnb5cqf52l31_n7r6zv3j6z80000gn/T/ipykernel_9970/1545779030.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c['hcpcs_0'] = c['hcpcs_ls'].str[0]


,billablePeriod_end,billablePeriod_start,contained_0_birthDate,contained_0_gender,contained_0_id,contained_0_identifier_0_type_coding_0_code,contained_0_identifier_0_type_coding_0_display,contained_0_identifier_0_value,contained_0_name_0_family,contained_0_name_0_given,...,drg_code,provider_number,national_provider_identifier,type_of_bill,claim_type,location_of_bill,diagnosis_ls,hcpcs_ls,procedure_ls,hcpcs_0


In [25]:
#c[c['unique_claim_ID'] == '-100007150']
c[c['unique_claim_ID'] == '100005174']

,billablePeriod_end,billablePeriod_start,contained_0_birthDate,contained_0_gender,contained_0_id,contained_0_identifier_0_type_coding_0_code,contained_0_identifier_0_type_coding_0_display,contained_0_identifier_0_value,contained_0_name_0_family,contained_0_name_0_given,...,drg_code,provider_number,national_provider_identifier,type_of_bill,claim_type,location_of_bill,diagnosis_ls,hcpcs_ls,procedure_ls,hcpcs_0
32504,2005-12-16,2005-12-16,NaN,female,patient,MC,Patient's Medicare Number,1S00E00AA50,Moore2,"[""L.""]",...,NaN,8888853578,NaN,NaN,professional,NaN,"[D649, E669, I10]",[],[],NaN


In [ ]:
# print(f"Patient - Medicare Number: {medicare_number}")
# print(f"Race: {p['race']}")
# print(f"Gender: {p['gender']}")
# print(f"Birth Date: {p['birthDate']}")
# print(f"Deceased: {p['deceasedBoolean'] == 1}")
# print(f"Postal Code: {p['address_0_postalCode']}")